In [2]:
import os #Os Library
from keras.layers.normalization import BatchNormalization #Keras
import gc  # Gabage collector for cleaning deleted data from memory
from keras import Sequential #Keras
from keras import backend as K #Keras
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dropout, Dense #Keras
import keras #Keras
import cv2 #OpenCV for reading and resizing images
import numpy as np  #Numpy
import pandas as pd #For reading CSV File 

Using TensorFlow backend.


In [3]:
df = pd.read_csv('styles.csv',usecols=['id','masterCategory']) #Read CSV File 

In [4]:
df.masterCategory.value_counts() #Value Counts of each category

Apparel           21400
Accessories       11289
Footwear           9222
Personal Care      2404
Free Items          105
Sporting Goods       25
Home                  1
Name: masterCategory, dtype: int64

In [7]:
df.masterCategory.value_counts().index

Index(['Apparel', 'Accessories', 'Footwear', 'Personal Care', 'Free Items',
       'Sporting Goods', 'Home'],
      dtype='object')

In [8]:
X = [] #We will store image in it
y = [] #W will Store Category here
allImages = os.listdir('images/') #Read all images inside images folder
for image in allImages: #Loop 
    if df.loc[df['id'] == int(image.split(".")[0])].shape == 0: #Check if the ID exist in the Dataframe of the image
        continue
    
    img = cv2.imread('images/'+image) #Read the image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Grayscale it. Color do not matter in this case
    img = cv2.resize(img, (60,60), interpolation = cv2.INTER_AREA)   #Resize all images to 60x60
    X.append(img) #Append the image
    y.append( df.loc[df['id'] == int(image.split(".")[0])].iloc[0]['masterCategory'] ) #Append the category

In [9]:
X = np.array(X) #Convert images to Numpy array
X = np.array(X).reshape(X.shape[0],X.shape[1],X.shape[2],1) #Add one extra channel/depth dimension
y = np.array(y) #Convert Y  to numpy array
print(X.shape)
print(y.shape)

(44441, 60, 60, 1)
(44441,)


In [10]:
y = pd.DataFrame(data=y) #Convert Y to pandas dataframe
y = pd.get_dummies(y) #Apply one hot encoding 
y

,0_Accessories,0_Apparel,0_Footwear,0_Free Items,0_Home,0_Personal Care,0_Sporting Goods
0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0
5,0,1,0,0,0,0,0
6,0,1,0,0,0,0,0
7,0,1,0,0,0,0,0
8,0,1,0,0,0,0,0
9,0,1,0,0,0,0,0


In [11]:
y.columns

Index(['0_Accessories', '0_Apparel', '0_Footwear', '0_Free Items', '0_Home',
       '0_Personal Care', '0_Sporting Goods'],
      dtype='object')

In [ ]:

y = np.array(y) #Convert back to Numpy array
y.shape

In [ ]:
from sklearn.model_selection import train_test_split #Used for splitting data into train and test
train, test, target, target_test = train_test_split(X, y, test_size=0.2, random_state=42) #Split data intot 80% and 20% ratio.

In [ ]:
# We will use a batch size of 32. Note: batch size should be a factor of 2.***4,8,16,32,64...***
import keras.backend as K
batch_size = 32

# Lets create the augmentation configuration
# This helps prevent overfitting, since we are using a small dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                                    rotation_range=40, #Rotate the image by 40 degree
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2, #Zoom by 20%
                                    horizontal_flip=True, #Horizontal flip
                                    fill_mode='nearest')

val_datagen = ImageDataGenerator()  # We do not augment validation data. we only perform rescale

# Create the image generators
train_generator = train_datagen.flow(train, target,batch_size=batch_size)
val_generator = val_datagen.flow(test, target_test, batch_size=batch_size)

In [ ]:
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    #Input Convolutional Layer. Filter of 32 and Kernel Size of 3x3
    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    #ReLU activation function
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3))) #Pooling of 3x3
    model.add(Dropout(0.25)) #Dropout of 0.25 to avoid overfitting

    #Second Convolutional Layer. Filter of 64 and Kernel Size of 3x3
    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu")) #ReLU activation function
    model.add(BatchNormalization(axis=chanDim))
    
    #Third Convolutional Layer. Filter of 64 and Kernel Size of 3x3
    model.add(Conv2D(64, (3,3), padding="same")) 
    model.add(Activation("relu")) #ReLU activation
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2))) #Pool size of 2x3
    model.add(Dropout(0.25)) #Dropout of 0.25 to avoid overfitting

    #Fourth Convolutional Layer. Filter of 128 and Kernel Size of 3x3
    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu")) #ReLU activation
    model.add(BatchNormalization(axis=chanDim))
    
    #Fifth Convolutional Layer. Filter of 128 and Kernel Size of 3x3
    model.add(Conv2D(128, (3,3), padding="same")) 
    model.add(Activation("relu")) #ReLU activation
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))  #Pool size of 2x3
    model.add(Dropout(0.25)) #Dropout of 0.25 to avoid overfitting

    model.add(Flatten()) #Flatten the layer
    model.add(Dense(128)) #Hidden layer of size 128
    model.add(Activation("relu")) #ReLU activation
    model.add(BatchNormalization())
    model.add(Dropout(0.4)) #Dropout of 0.4 to avoid overfitting

    model.add(Dense(classes)) #Define Number of classes
    model.add(Activation("softmax")) #Softmax activation function to output layer

    return model


model = build(width=60, height=60, depth=1, classes=7) #Call Model


#Build Model
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam', metrics = ['acc'])


from keras.callbacks import ModelCheckpoint
early_stop = keras.callbacks.EarlyStopping(patience=100) #If accuracy do not improve for 100 rounds stop the training
checkpointer = ModelCheckpoint(filepath='categoryModel.h5', verbose=1, save_best_only=True) #Save the best model only
callbacks = [checkpointer,early_stop]  #Define callbacks

In [ ]:
history = model.fit_generator(train_generator, #Train the model
                              steps_per_epoch=32,
                              epochs=500, #Train model for 500 epochs/rounds
                              validation_data=val_generator,
                              validation_steps=32, callbacks=callbacks)

In [ ]:
model.load_weights('categoryModel.h5') #Load best epoch weights
predict = model.predict(test) #Predict
predict = np.argmax(predict,axis=1) #Get class index with highest probablity
print("Accuracy", accuracy_score(np.argmax(target_test,axis=1), predict)) #Print accuracy

In [ ]:
import pickle #Pickle File
pickle.dump(model, open('categoryModel', 'wb')) #Save Model in Pickle Format also